# WORK IN PROGRESS

Use Beautify.ipynb instead

In [ ]:
%matplotlib inline
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFilter
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import scipy.stats as st

In [ ]:
import torch.utils.data
from net import *
from model import Model
from launcher import run
from checkpointer import Checkpointer
from dlutils.pytorch import count_parameters
from defaults import get_cfg_defaults
import lreq
import logging
from torch.autograd import Variable

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import face_recognition
from tqdm import trange
import PIL
from ffhq_dataset.landmarks_detector import LandmarksDetector
import bz2
from keras.utils import get_file


In [ ]:
path = "raw_images/"
valid_images = (".jpg",".png")
for f in os.listdir(path):
    if f.endswith(valid_images):
        img = PIL.Image.open(path + f)
        display(img.resize((int(img.size[0]/3), int(img.size[1]/3)), PIL.Image.LANCZOS))

In [ ]:
!python align_images.py raw_images/ aligned_images/ alignement_vector/

In [ ]:
path = "aligned_images/"
for f in os.listdir(path):
    if f.endswith(valid_images):
        img = PIL.Image.open(path + f)
        display(img.resize((256,256), PIL.Image.LANCZOS))

In [ ]:
cfg = get_cfg_defaults()
#cfg.OUTPUT_DIR = "training_artifacts/ffhq"
cfg.merge_from_file("configs/ffhq.yaml")

logger = logging.getLogger("logger")
logger.setLevel(logging.DEBUG)

model = Model(
    startf=cfg.MODEL.START_CHANNEL_COUNT,
    layer_count=cfg.MODEL.LAYER_COUNT,
    maxf=cfg.MODEL.MAX_CHANNEL_COUNT,
    latent_size=cfg.MODEL.LATENT_SPACE_SIZE,
    truncation_psi=cfg.MODEL.TRUNCATIOM_PSI,
    truncation_cutoff=cfg.MODEL.TRUNCATIOM_CUTOFF,
    mapping_layers=cfg.MODEL.MAPPING_LAYERS,
    channels=cfg.MODEL.CHANNELS,
    generator=cfg.MODEL.GENERATOR,
    encoder=cfg.MODEL.ENCODER)

#model = torch.load("training_artifacts/ffhq/model_submitted.pth")
#checkpoint = torch.load("training_artifacts/ffhq/model_submitted.pth")

#print(checkpoint["models"].keys())
#model.load_state_dict(checkpoint["models"])

model.eval()
model.requires_grad_(False)


decoder = model.decoder
encoder = model.encoder
mapping_tl = model.mapping_d
mapping_fl = model.mapping_f
dlatent_avg = model.dlatent_avg

logger.info("Trainable parameters generator:")
count_parameters(decoder)

logger.info("Trainable parameters discriminator:")
count_parameters(encoder)

arguments = dict()
arguments["iteration"] = 0

model_dict = {
    'discriminator_s': encoder,
    'generator_s': decoder,
    'mapping_tl_s': mapping_tl,
    'mapping_fl_s': mapping_fl,
    'dlatent_avg': dlatent_avg
}

checkpointer = Checkpointer(cfg,
                            model_dict,
                            {},
                            logger=logger,
                            save=False)

extra_checkpoint_data = checkpointer.load()

model.eval()

layer_count = cfg.MODEL.LAYER_COUNT

In [ ]:
def encode(x):
    Z, _ = model.encode(x, layer_count - 1, 1)
    Z = Z.repeat(1, model.mapping_f.num_layers, 1)
    return Z

def decode(x):
    return model.decoder(x, layer_count - 1, 1, noise=True)

In [ ]:
def load(path):
    img = np.asarray(Image.open(path))

    if img.shape[2] == 4:
        img = img[:, :, :3]
    im = img.transpose((2, 0, 1))
    x = torch.tensor(np.asarray(im, dtype=np.float32), requires_grad=True) / 127.5 - 1.
    if x.shape[0] == 4:
        x = x[:3]

    needed_resolution = model.decoder.layer_to_resolution[-1]
    while x.shape[2] > needed_resolution:
        x = F.avg_pool2d(x, 2, 2)
    if x.shape[2] != needed_resolution:
        x = F.adaptive_avg_pool2d(x, (needed_resolution, needed_resolution))

    img_src = ((x * 0.5 + 0.5) * 255).type(torch.long).clamp(0, 255).cpu().type(torch.uint8).transpose(0, 2).transpose(0, 1).numpy()

    latents_original = encode(x[None, ...])
    latents = latents_original[0, 0].clone()
    latents -= model.dlatent_avg.buff.data[0]

    return latents, latents_original, img_src

In [ ]:
def convert_to_image(latent_vector):
    x_rec = decode(latent_vector.detach())
    resultsample = ((x_rec * 0.5 + 0.5) * 255).type(torch.long).clamp(0, 255)
    resultsample = resultsample.cpu()[0, :, :, :]
    
    return (resultsample.type(torch.uint8).transpose(0, 2).transpose(0, 1))


def gkern(kernlen=21, nsig=3):
    """Returns a 2D Gaussian kernel."""

    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()

In [ ]:
image_path = "aligned_images/IMG_5551_02.png"

latents, latents_original, img_src = load(image_path)

img = Image.fromarray(convert_to_image(latents_original).detach().numpy())
display(img.resize((256,256), PIL.Image.LANCZOS))

In [ ]:
input_latent = Variable(latents_original.data, requires_grad=True)

if img_src.shape[2] == 4:
    img_src = img_src[:, :, :3]
img = img_src.transpose((2, 0, 1))
x = torch.tensor(np.asarray(img, dtype=np.float32), requires_grad=True) / 127.5 - 1.
if x.shape[0] == 4:
    x = x[:3]
x = x.reshape(1,3,1024,1024)
x = Variable(x, requires_grad=False)

t = trange(10, desc='Loss', leave=True)
for _ in t:
    # Foreward pass
    prediction = model.decoder(input_latent, layer_count - 1, 1, noise=False)

    loss = ((prediction - x).pow(2) * torch.tensor(gkern(1024, 2) * 5000)).sum()
    
    # Backward pass
    loss.backward()
    
    t.set_description("Loss: " + str(loss.detach().numpy()))

    # Update
    input_latent = input_latent - 0.004 * input_latent.grad.data
    input_latent = Variable(input_latent.data, requires_grad=True)

img = Image.fromarray(convert_to_image(input_latent).detach().numpy())
display(img.resize((256,256), PIL.Image.LANCZOS))

In [ ]:
face_landmarks = "alignement_vector/IMG_5551_02.npy"
src_file = "raw_images/IMG_5551.jpg"
mask_file = "masks/IMG_5551_02.png"

In [ ]:
latents_x = input_latent.detach().clone()

indices = [0, 1, 2, 3, 4, 10, 11, 17, 19]
W = [torch.tensor(np.load("principal_directions/direction_%d.npy" % i), dtype=torch.float32) for i in indices]

def alter(gender, smile, attract, hair, age, lips, nose, fat, glasses):
    altered = input_latent.detach().clone()
    
    altered += gender *  W[0]
    altered += smile *   W[1]
    altered += attract * W[2]
    altered += hair *    W[3]
    altered -= age *     W[4]
    altered += lips *    W[5]
    altered += nose *    W[6]
    altered += fat *     W[7]
    altered += glasses * W[8]
    
    im = convert_to_image(altered)
    
    img = Image.fromarray(im.detach().numpy())
    
    landmarks_detector = LandmarksDetector(landmarks_model_path)
    generated_image_landmarks = face_recognition.face_landmarks(np.array(img))[0]
        
    lm = np.load(face_landmarks)
    lm_chin = lm[0:17]  # left-right
    lm_eyebrow_left = lm[17:22]  # left-right
    lm_eyebrow_right = lm[22:27]  # left-right
    lm_nose = lm[27:31]  # top-down
    lm_nostrils = lm[31:36]  # top-down
    lm_eye_left = lm[36:42]  # left-clockwise
    lm_eye_right = lm[42:48]  # left-clockwise
    lm_mouth_outer = lm[48:60]  # left-clockwise
    lm_mouth_inner = lm[60:68]  # left-clockwise

    # Calculate auxiliary vectors.
    eye_left = np.mean(lm_eye_left, axis=0)
    eye_right = np.mean(lm_eye_right, axis=0)
    eye_avg = (eye_left + eye_right) * 0.5
    eye_to_eye = eye_right - eye_left
    mouth_left = lm_mouth_outer[0]
    mouth_right = lm_mouth_outer[6]
    mouth_avg = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg

    left_eye_before = np.mean(generated_image_landmarks["left_eye"], axis=0)
    right_eye_before = np.mean(generated_image_landmarks["right_eye"], axis=0)

    x1b, y1b = left_eye_before
    x2b, y2b = right_eye_before

    y1a, x1a = eye_left
    y2a, x2a = eye_right

    # Generate linear system to compute transformation
    a = np.matrix([[-y1b, x1b, 0, 1], [x1b, y1b, 1, 0], [-y2b, x2b, 0, 1], [x2b, y2b, 1, 0]])

    b = np.array([y1a, x1a, y2a, x2a])

    # Solve linear system
    x = np.linalg.solve(a, b)

    # Calc transformation matrix
    transform_matrix = np.matrix([[x[1], -x[0], x[3]], [x[0], x[1], x[2]], [0, 0, 1]])

    transform_matrix = np.array([[x[1], -x[0], x[3]], [x[0], x[1], x[2]], [0, 0, 1]])

    transform_inv = np.linalg.inv(transform_matrix)

    background = Image.open(src_file).convert("RGB")

    foreground = img
    mask = Image.open(mask_file).convert("L").resize(foreground.size).filter(ImageFilter.GaussianBlur(20))
    
    foreground = foreground.transform(background.size, PIL.Image.AFFINE, transform_inv.flatten()[:6], resample=Image.NEAREST)    
    mask = mask.transform(background.size, PIL.Image.AFFINE, transform_inv.flatten()[:6], resample=Image.NEAREST)

    
    background = Image.composite(foreground, background, mask)
    
    size = 512, 512
    background.thumbnail(size, Image.ANTIALIAS)
    
    display(background)

    
w = interact(
    alter, 
    gender=(-19,20,1), 
    smile=(-9,10,1), 
    attract=(-9,10,1),
    hair=(-9,10,1),
    age=(-9,10,1), 
    lips=(-9,10,1),
    nose=(-9,10,1),
    fat=(-9,10,1),
    glasses=(-99,100,1)
)
display(w)